In [45]:
import zlib
import math
import nltk
import itertools
import string
import random
import base64
import unicodedata

In [70]:
a = zlib.compress(b'apples and oranges')
# a = zlib.compress(b'a')
# a[4] = 's'
byteorder = 'big'
# a = int.from_bytes(a, 'little') << 2
num_bytes = len(a)
a = int.from_bytes(a, byteorder)# + (2 ** 20)
print(a)
a = a.to_bytes(num_bytes, byteorder)
# a = str(a).encode()
print(zlib.decompress(a))

b = zlib.compress(b'plums and pears')

193813636406069426573129083747198422080663845634188243454133992
b'apples and oranges'


In [142]:
z = zlib.compressobj(level=9)

In [150]:
def int_comp(x):
    return int.from_bytes(
        zlib.compress(bytes(x, 'UTF-8')),
        byteorder
    )
ic = int_comp

In [151]:
text = 'I have neither a fear, nor a presentiment, nor a hope of death. Why should I? With my hard constitution and temperate mode of living, and unperilous occupations, I ought to, and probably shall, remain above ground till there is scarcely a black hair on my head. And yet I cannot continue in this condition!'

In [159]:
m = ic(text)
n = ic(text+' t')
print(m, n, m-(n*(10**round(len(str(m/n))))))

89969844122252241980381186095195172050569469965151240149639482176962088838920190096558819960905325208905812217904950943953502113426378303803304642506777097908971930726065173189352922458236329566791322186763510428348178170043958489050939420836046743665324658416259699581420836252112596159038647597305887914111972643555296082768712297263344701805744543073870101797328830763394214566635464365918376471246758956528495121523947620057892375527748000094549092698331954337020221378631585499856530758 23032280095296573946977583640369964044945784311078717478307707437302294742763568664719057909991763253479887927783667441652096541037152845773645988481734937064696814265872684336474348149308500369098578479811458669657133611531253373197040491734027966378323112554562483092843734080540824616713893784910307306012664996750155797188790348099416243662270603026910746060116180675428918929058678877675104376639170292871294751110130590734820448135103488024204567730772980310277176672929685871698782678490 -230322800952

In [2]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\diamo\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
hamlet = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')
# hamlet = ' '.join(words)
print(hamlet[:100])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo a


In [80]:
chars = string.punctuation + string.digits# + string.ascii_letters
chars = list(chars) + [chr(n) for n in range(161, 256)]
chars = set(chars) - set(['"', "'", '\\', '&'])
replacements = []
for l in range(1, 3):
    spans = [''.join(c) for c in itertools.combinations_with_replacement(chars, r=l)]
    random.shuffle(spans)
    replacements.extend(spans)
print(len(replacements))
print(replacements[:20])
# random.shuffle(replacements)


9044
['È', ';', '6', '¡', '~', 'Ý', '+', '¢', '¥', 'Ö', '¨', 'ü', '(', 'ù', '%', 'ô', 'î', 'ª', 'ø', '¾']


In [6]:
# replacements[:100]

In [56]:
def compress(input_text, log=False, window_size=(2, 10, 1), separator='&'):
    sections = {}
    w = 0
    reps = []
    checked = []
    
    original = input_text
    compressed = input_text
    
    for k in range(5):
        print(f'Compression pass {k+1}')
        for i in range(*window_size):
            if log:
                print(f'Scanning sequences of length {i}')
            for j in range(0, len(compressed)-i):
                window = compressed[j:j+i]
                if window and window not in checked and compressed.count(window) > 1 and window not in sections.values():
                    for rep in replacements:
                        if rep not in compressed and len(window) > len(rep)+len(separator):
                            sections[w] = window
                            reps.append(rep)
                            w += 1
                #             print(window)
        #                     r = f'[{w}]'
                            compressed = compressed.replace(window, rep+separator)
                            break
                checked.append(window)
            if log:
                print(len(compressed))
    return compressed, reps, sections

In [57]:
def decompress(input_text, reps, sections, separator='&'):
    for i, r in enumerate(reps[::-1]):
        input_text = input_text.replace(r+separator, sections[len(reps)-i-1])
    return input_text

In [103]:
argdict = {
    'separator': ''
}
a, r, s = compress('grapes, bananas, apples, peaches, and bananas', window_size=(2, 10, 1), **argdict)
print(a)
print(decompress(a, r, s, **argdict))

Compression pass 1
Compression pass 2
Compression pass 3
Compression pass 4
Compression pass 5
grÈ+¡6Èpl+peach+and ¡
grapes, bananas, apples, peaches, and bananas


In [106]:
decompress('grÈ+¡6Èpl+peach+and ¡', r, s, **argdict)

'grapes, bananas, apples, peaches, and bananas'

In [108]:
s = 500
h = hamlet[:1000]
wa = (2, 20, 1)
wb = (10, 2, -1)
compressed, reps, sections = compress(h, window_size=wa, log=True, **argdict)
original = h
print(original[:s], '\n'*4, compressed[:], '\n'*4)
print(decompress(compressed, reps, sections, **argdict))

Compression pass 1
Scanning sequences of length 2
456
Scanning sequences of length 3
442
Scanning sequences of length 4
442
Scanning sequences of length 5
442
Scanning sequences of length 6
442
Scanning sequences of length 7
442
Scanning sequences of length 8
442
Scanning sequences of length 9
442
Scanning sequences of length 10
442
Scanning sequences of length 11
442
Scanning sequences of length 12
442
Scanning sequences of length 13
442
Scanning sequences of length 14
442
Scanning sequences of length 15
442
Scanning sequences of length 16
442
Scanning sequences of length 17
442
Scanning sequences of length 18
442
Scanning sequences of length 19
442
Compression pass 2
Scanning sequences of length 2
430
Scanning sequences of length 3
430
Scanning sequences of length 4
430
Scanning sequences of length 5
430
Scanning sequences of length 6
430
Scanning sequences of length 7
430
Scanning sequences of length 8
430
Scanning sequences of length 9
430
Scanning sequences of length 10
430
Scanni

In [344]:
enc = 'UTF-8'
c = zlib.compress(original.encode(enc))#.decode(enc)
base64.b64encode(c)

b'eJxVk0tv20AMhO/6FTw1F1dADz0kl8B9JA6atEVtIAiKHpgVJS2yWrr7iOt/31nJD+UiSFpqOPw4+r3phTaBO2mskLa04sFJouc9PVrnLA+07vlF4lY4CH34eHn5p6qqpUk50s9ghxxrWhsVz+Mj11X11ScJ9ImD59AosW/oJrA3NhqltFP6LD5ZLy6imE6FNT32ehEp9RLkGgflo5q+8x4ScQfJQa5onYreO3r1rbqm2msOFMW1UqSKVk336jtyNkuRom/Wd+PZJHfsdl0dqlcyO33STEYHoUFjIoOR2+zcnl636mns1eMy63WxsZG87iimoPqC8cRlWVAHhugOC0rPMgMwa3ajATX4PogTadE0mx5v2IP31ST9bFOBaTDrolpi8jvCSqI1L0KcqMdW0tENJFeMqeGTem7ob7ZwcZsx7gynJmJ60BwFnm0IRzyTwKM4B/eqjbddn2q6a0c97HEQqK00cLLjUqsHDgblOS5G0L9sZhdLhgaEh5PpF7DflLOBBmQIpmKqD/GYCdFJaMzD5POuoAEI3PRj9IpMPa3/inZvkwIx4AxWfBML8ZFqFzTDZZntoayq0LsX28n7QfxUJfSF/Xz/t/bArxCgEcFZ4Ae1HLBfZMAL4rHGUqyERXGD2VJfFmklXzRF4oD8nDiUwDPYbB0bJLmb95po49/5Z9PkpfQd267UOT6pnKO35n1pjRRg2CPOt0iWtBUx5b/+D2AfREs='

In [67]:
reconstructed = compressed
# u = random.randint(0, len(reconstructed))
# reconstructed = reconstructed[:u] + '?' + reconstructed[u:]

In [124]:
def detach(text, x, f=''):
    return text[:x] + f + text[x+1:]

variations = []
for i in range(10):
    newstring = compressed
    for j in range(random.randint(1, 400)):
        c = random.choice(list(chars)+['']*2)
        newstring = detach(newstring, random.randint(0, len(newstring)), c)
    variations.append(newstring)

reconstructed = variations[0]

print(decompress(reconstructed[:s], reps, sections, **argdict))
# todo: sort by frequency

[

   Fran.. W Tue yor die ou Hreue yoe m. H Franciscohellcham 
Ahakespearam ht5  Hois    Fran.


Ape

  ur g

Marcee olene m Prim Mar.

Enter . nardo ay W StandBaro a Mar Mareniue yo yoolam  Barnardo. Wu rape?
  Fran. N. Wy antiweoo Franciscoli Stand & Svu pe SonMarcellyour s?
  Fy W the

   t L?
  Fran. Nam coli Standthee ke  tge   Fran. Barnardo. Aam Bar.5y    Fran.  td iue yo aliraohocoe mtiralllehtd on your houdo

   Bar. 'Tis is , w ourook t Prim aranyoget thee g

o beteran.  Fo Barnardothis releene mus.r  tlsy Wdoiue yoy W:tst Primlid,pend I Bar Franciscoarce  Horatio au goohearhear Barngooue yoMarcellhad dnightoliey ran FranciscoardEnot ou Barnous  wetirring

of Barn. Wendue goora. oune yo:u Barmeete 9riam. Ater  the o aiuals am .ra  ch, bid them make  the styoI ue yoigr1nd dnightou

dnightran. me th.

ke geheare iue yohtndus.y W ranho's th. Fri]ds. Hmeis ke hayond

  ne. A t1ey Wgeur m. Hha the Daen Francisco Standpeu gooam nofhtt

   March  F lemewel.

 neig tooldier, hlehath

In [208]:
len(hamlet)

162881